In [1]:
import csv
import sys

import numpy as np
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

MIN_TRANSFORMERS_VERSION = "4.25.1"

# check transformers version
assert (
    transformers.__version__ >= MIN_TRANSFORMERS_VERSION
), f"Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher."

root_dir = "/Users/kenzaamara/GithubProjects/syntax-shap/shap2"
sys.path.append(root_dir)

/Users/kenzaamara/opt/anaconda3/envs/shap/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
model.save_pretrained("/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|██████████| 2/2 [01:02<00:00, 31.36s/it]


In [14]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.save_pretrained("/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2")

('/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/tokenizer_config.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/special_tokens_map.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/vocab.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/merges.txt',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/added_tokens.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/gpt2/tokenizer.json')

: 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.save_pretrained("/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral")

('/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral/tokenizer_config.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral/special_tokens_map.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral/tokenizer.model',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral/added_tokens.json',
 '/Users/kenzaamara/GithubProjects/syntax-shap/models/mistral/tokenizer.json')

In [9]:
model_name = "gpt2"#"mistralai/Mistral-7B-v0.1"
# model = GPT2LMHeadModel.from_pretrained("gpt2") #
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# set model decoder to true
model.config.is_decoder = True
# set text-generation params under task_specific_params
model.config.task_specific_params["text-generation"] = {
    "do_sample": True,
    "max_new_tokens": 1,
    #"temperature": 0.7,
    #"top_k": 50,
    #"no_repeat_ngram_size": 2,
}

In [3]:
from models import TextGeneration
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lmmodel = TextGeneration(model, tokenizer, device=device)

In [4]:
lmmodel.inner_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [47]:
lmmodel("A boy is not a")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


array([[2933]])

In [7]:
input = (np.array(["Peter is a father with a"], dtype="<U10"),)
lmmodel(*input)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


array([[27449]])

In [49]:
lmmodel.inner_model.config.task_specific_params["text-generation"] = {
    "do_sample": True,
    "max_new_tokens": 1,
    "temperature": 0.7,
    "top_k": 50,
    "no_repeat_ngram_size": 2,
}

In [6]:
prompt = "A boy is not a"
tokenizer([prompt], return_tensors="pt")

{'input_ids': tensor([[  32, 2933,  318,  407,  257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [20]:
github_local_data_url = "/Users/kenzaamara/GithubProjects/shap/data/"
tsv_file = open(github_local_data_url + "Inconsistent-Dataset-Negation.tsv")
read_tsv = list(csv.reader(tsv_file, delimiter="\t"))
data = []
for row in read_tsv:
    data.append(row[1][:-8])
data = np.array(data)

In [21]:
import explainers

explainer = explainers.PartitionExplainer(lmmodel, lmmodel.tokenizer)
explainer([data[315]])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


self.X A boy is not a


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


self.X ['...']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


self.X ['Peter ... a father with a' '... is a father with a' 'Peter is ...'
 '... a father ...']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


self.X ['Peter is a ...' 'Peter is ... father ...' 'Peter ...' '... is ...'
 '... a ...' '... father ...']


.values =
array([[[  723.25 ],
        [-1037.   ],
        [  757.875],
        [ 1000.875],
        [ 1493.5  ],
        [ 1618.5  ]]])

.base_values =
array([[393.]])

.data =
(array(['Peter', ' is', ' a', ' father', ' with', ' a'], dtype=object),)

In [22]:
import numpy as np
import spacy
from utils import create_dataframe_from_tree, spacy_doc_to_tree


def get_dependency_dt(text):  # Example usage:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text + " MASK")

    # Convert Spacy dependency tree to a Tree object
    tree_root = spacy_doc_to_tree(doc)

    # Example usage with the Tree structure
    tree_df = create_dataframe_from_tree(tree_root)
    tree_df = tree_df[tree_df["token"] != "MASK"]
    return tree_df


def filter_invalid_inputs(data, tokenizer):
    invalid_ids = []
    invalid_inputs = []
    for id, input in enumerate(data):
        M = len(tokenizer.encode(input))
        dependency_dt = get_dependency_dt(input)
        max_pos = dependency_dt["position"].max() + 1
        if M != max_pos:
            invalid_ids.append(id)
            invalid_inputs.append(input)
    invalid_inputs = np.unique(invalid_inputs)
    return invalid_ids, invalid_inputs


# invalid_ids, invalid_inputs = filter_invalid_inputs(data, lmmodel.tokenizer)
# np.save('/Users/kenzaamara/GithubProjects/shap/results/invalid_ids.npy', invalid_ids)
# np.save('/Users/kenzaamara/GithubProjects/shap/results/invalid_inputs.npy', invalid_inputs)

In [ ]:
invalid_ids = np.load(
    "/Users/kenzaamara/GithubProjects/shap/results/invalid_ids.npy", allow_pickle=True
)
invalid_inputs = np.load(
    "/Users/kenzaamara/GithubProjects/shap/results/invalid_inputs.npy",
    allow_pickle=True,
)

In [ ]:
## Load the computed shapley values
import pickle

algorithms = ["partition"]  # , 'exact', 'dtree', 'r-dtree']
weighted = False
for algorithm in algorithms:
    if algorithm.endswith("dtree"):
        weighted = weighted
    else:
        weighted = False
    w_str = "_weighted" if weighted else ""
    print("w_str", w_str)
    filename = (
        f"/Users/kenzaamara/GithubProjects/shap/results/loss_dist/shap_values_{algorithm}"
        + w_str
        + ".pkl"
    )
    with open(filename, "rb") as f:
        shap_values = pickle.load(f)

    # Perprocessing
    data = data[0:314]
    scores = shap_values.values[0:314]

    ## filter the invalid inputs
    filtered_data = np.delete(data, invalid_ids[invalid_ids < 314], axis=0)
    filtered_scores = np.delete(scores, invalid_ids[invalid_ids < 314], axis=0)
    assert len(filtered_data) == len(filtered_scores)

In [ ]:
from evaluation import aopc_fidelity

fid_scores = aopc_fidelity(filtered_data, filtered_scores, lmmodel)
print("Fidelity score average: ", fid_scores.mean())
kl_div_mean, kl_div_batchmean, kl_fid_scores = kl_divergence_fidelity(
    filtered_data, filtered_scores, lmmodel
)
kl_div_mean, kl_div_batchmean, kl_fid_scores

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


def plot_shap_values(shap_values, title, input_id=0):
    # Extracting data for plotting
    values = shap_values.values.flatten()  # Flatten the nested list
    features = shap_values.data[0]

    # Creating a DataFrame
    df = pd.DataFrame({"Feature": features, "Shapley Value": values})
    # Determine color for each bar individually
    colors = ["red" if x > 0 else "blue" for x in df["Shapley Value"]]

    # Plotting
    fig = plt.figure(figsize=(4, 4))
    sns.barplot(data=df, x="Feature", y="Shapley Value", palette=colors, legend=False)
    plt.title(title)
    plt.xlabel("Shapley Value")
    plt.ylabel("Feature")

    plt.savefig(root_dir + f"/figures/{title}_id_{input_id}.png", bbox_inches="tight")
    plt.show()


plt.plot(explainer, fid_scores)